# Анализ клиентской базы

Перед нами стоит задача провести анализ клиентской базы и выявить наиболее ценных и перспективных клиентов и клиентские сегменты. 

Пусть вас не пугает много кода. Строчки и занчения которые надо изменять я укажу через комментарии `#!!! надо изменить`. Также важно, чтобы структура вашего файла соответстовала оригинальному. 

## Загрузим библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

## Прочитаем данные из файла

Желательно чтобы в файле была всего одна вкладка. Структура должна соотчетствовать. 

Поля:
- Name имя клиента
- Date дата покупки
- Revenue - выручка
- Cost - себестоимость

In [ ]:
df=pd.read_excel('SimpleBase.xlsx') # !!! измените название файла на свое
df.sample(3, random_state=46) #три случайные строки

In [ ]:
df.loc[1, 'Date']

In [ ]:
# добавим поле с прибылью от продажи
df['Profit']=df['Revenue']-df['Cost']
df['ProfitPct']=df['Profit']/df['Revenue'] #рентабельность

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(18,10), bins=100);

In [ ]:
df['Revenue'].hist(figsize=(18,4), bins=100);

In [ ]:
df[df['Revenue']>200000] #!!! изменить значение в фильтре согласно Вашим параметрам

In [ ]:
df[df['Revenue']<30000]['Revenue'].hist(figsize=(18,4), bins=100); # !!!ихменить значение в фильтре согласно вашим параметрам

## Сводные данные по клиентам

Ниже данные из журнала покупок преобразуем в сводные данные. 

Выполняем группировку.

`Total_sale` - общее количество покупок клиента за все время.

In [ ]:
df_clients=df.groupby('Name').agg({'Revenue':'sum', 'Profit':'sum', 'Date':'count'}).reset_index()
df_clients.rename(columns={'Date':'Total_sale'}, inplace=True) #переименуем столбец
df_clients.sample(3)

In [ ]:
df_clients.describe()

In [ ]:
df_clients[df_clients['Revenue']>3000000] #!!! изменяйте значение фильтра

In [ ]:
df_clients[df_clients['Total_sale']>300] #!!! изменяйте значение фильтра

In [ ]:
df_clients[(df_clients['Revenue']<500000) &
          (df_clients['Profit']<300000) &
          (df_clients['Total_sale']<150)].hist(figsize=(18,6), bins=50); #!!! изменяйте значение фильтра

## Коэффициент удержания клиентов

Показывает какой процент клиентов мы теряем по годам.

Подготовим данные.

- 'start_date' первая покупка, 
- 'last_date' последняя покупка,
- 'start_year' первый год,
- 'last_year' последний год

In [ ]:
df_t=df.groupby('Name').agg({'Date':['min', 'max']}).reset_index()
df_t.columns=['Name', 'start_date', 'last_date']

if 'start_date' in df_clients.columns:
    df_clients.drop(columns=['start_date', 'last_date'], inplace=True)

df_clients=pd.merge(df_clients, df_t, on='Name')

df_clients['start_year']=df_clients['start_date'].apply(lambda x: x.year)
df_clients['last_year']=df_clients['last_date'].apply(lambda x: x.year)
df_clients.head()

### Показатели удержания по годам

In [ ]:
ya=df_clients['start_year'].unique()
ya.sort()

all_r=[]
for i in range(0,len(ya)):
    start=len(df_clients[(df_clients['start_year']==ya[i])])
    t=[0 for x in range(i)]
    for j in range(i,len(ya)):
        t.append(round(len(df_clients[(df_clients['start_year']==ya[i]) & (df_clients['last_year']>=ya[j])])/start,2))
    all_r.append(t)
data=pd.DataFrame(all_r, columns=ya, index=ya)
data[data==0]=np.nan

plt.figure(figsize = (4,4))
plt.title('Когортный анализ: удердание')
sns.heatmap(data = data, 
            annot = True, 
            fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.8,
            cmap = "YlGnBu")
plt.show()

In [ ]:
data

### Когрты количество клиентов по годам

In [ ]:
all_r=[]
for i in range(0,len(ya)-1):
    t=[0 for x in range(i)]
    for j in range(i,len(ya)):
        t.append(len(df_clients[(df_clients['start_year']==ya[i]) & (df_clients['last_year']>=ya[j])]))
    all_r.append(t)
data=pd.DataFrame(all_r, columns=ya, index=ya[:-1])
data[data==0]=np.nan

plt.figure(figsize = (5,5))
plt.title('Когортный анализ: количество клиентов')
sns.heatmap(data = data, 
            annot = True, 
            fmt = '.0', 
            vmin = 0.0,
            vmax = 0.8,
            cmap = "YlGnBu")
plt.show()

In [ ]:
data

### Когорты по кварталам. Удержание

In [ ]:
df_clients['MinPurchaseQuarter'] = df_clients['start_date'].map(lambda date: 10*date.year + date.quarter)
df_clients['MaxPurchaseQuarter'] = df_clients['last_date'].map(lambda date: 10*date.year + date.quarter)

my_all=[]
min_y=df_clients['start_year'].min()
max_y=df_clients['last_year'].max()
max_date=df_clients['MaxPurchaseQuarter'].max()
for y in range(min_y, max_y+1):
    for m in range(1,5):
        nl=y*10+m
        if nl>max_date: break
        my_all.append(nl)
        
all_r=[]
for i in range(len(my_all)):
    start=len(df_clients[(df_clients['MinPurchaseQuarter']==my_all[i])])
    if start==0: start=1
    t=[0 for x in range(i)]
    for j in range(i,len(my_all)):
        t.append(round(len(df_clients[(df_clients['MinPurchaseQuarter']==my_all[i]) & 
                                   (df_clients['MaxPurchaseQuarter']>=my_all[j])])/start,2))
    all_r.append(t)
data=pd.DataFrame(all_r, columns=my_all, index=my_all)

data[data==0]=np.nan

plt.figure(figsize = (11,9))
plt.title('Когорты: удержание по кварталам')
sns.heatmap(data = data, 
            annot = True, 
            fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.8,
            cmap = "YlGnBu")
plt.show()

### Когорты: количество клиентов по кварталам

In [ ]:
all_r=[]
for i in range(len(my_all)):
    if start==0: start=1
    t=[0 for x in range(i)]
    for j in range(i,len(my_all)):
        t.append(len(df_clients[(df_clients['MinPurchaseQuarter']==my_all[i]) & 
                                   (df_clients['MaxPurchaseQuarter']>=my_all[j])
                                 ]))
    all_r.append(t)
data=pd.DataFrame(all_r, columns=my_all, index=my_all)

data[data==0]=np.nan

plt.figure(figsize = (11,9))
plt.title('Когорный анализ: количество клиентов')
sns.heatmap(data = data, 
            annot = True, 
            #fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.8,
            cmap = "YlGnBu")
plt.show()

### Когорты: выручка по кварталам

In [ ]:
all_r=[]
for i in range(len(my_all)):
    if start==0: start=1
    t=[0 for x in range(i)]
    for j in range(i,len(my_all)):
        t.append(df[df['Name'].isin(df_clients[(df_clients['MinPurchaseQuarter']==my_all[i]) & 
                                   (df_clients['MaxPurchaseQuarter']>=my_all[j])]['Name'])]['Revenue'].sum())
    all_r.append(t)
data=pd.DataFrame(all_r, columns=my_all, index=my_all)

data[data==0]=np.nan

plt.figure(figsize = (11,9))
plt.title('Когорный анализ: выручка')
sns.heatmap(data = data, 
            annot = True, 
            #fmt = '.0%', 
            vmin = 0.0,
            vmax = 0.8,
            cmap = "YlGnBu")
plt.show()

In [ ]:
data

In [ ]:
#подготовим данные и переменные для следующих анализов
maxDate=df_clients['last_date'].max()
df_clients['Days_long']=df_clients.apply(lambda x: (x['last_date']-x['start_date']).days, axis=1)

# RFM

RFM аббревиатура (англ. Recency Frequency Monetary — давность, частота, деньги) — сегментация клиентов в анализе сбыта по лояльности.

Определяет три группы:

- Recency (давность) — давность сделки, чем меньше времени прошло с момента последней активности клиента, тем больше вероятность, что он повторит действие
- Frequency (частота) — количество сделок, чем больше каких-либо действий совершит клиент, тем больше вероятность того, что он его повторит в будущем
- Monetary (деньги) — сумма сделок, чем больше денег было потрачено, тем больше вероятность того, что он сделает заказ

Сегментируем нашу базу.

In [ ]:
df_RFM=df_clients[['Total_sale', 'Revenue']].copy()
df_RFM['Recency']=df_clients['last_date'].apply(lambda x: (maxDate-x).days)
df_RFM.columns=['Frequency', 'Monetary', 'Recency']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

In [ ]:
df_RFM['Frequency'].value_counts()

In [ ]:
df_RFM['R'] = pd.qcut(df_RFM['Recency'].rank(method='first'), 5, [5,4,3,2,1])
df_RFM['F'] = pd.qcut(df_RFM['Frequency'].rank(method='first'), 5, [1,2,3,4,5])
df_RFM['M'] = pd.qcut(df_RFM['Monetary'].rank(method='first'), 5, [1,2,3,4,5])

df_RFM['RFM Score'] = np.array(df_RFM['R'].map(str)) + np.array(df_RFM['F'].map(str)) + np.array(df_RFM['M'].map(str))
df_RFM.head()

In [ ]:
segt_map = {
    r'[1-2][1-2]': 'потерянные',
    r'[1-2][3-4]': 'риск потерять',
    r'[1-2]5': 'не терять',
    r'3[1-2]': 'спящие',
    r'33': 'нужно внимание',
    r'[3-4][4-5]': 'лояльные',
    r'41': 'многообещающие',
    r'51': 'новые',
    r'[4-5][2-3]': 'потенциально лояльные',
    r'5[4-5]': 'чемпионы'
}

df_RFM['Segment'] = np.array(df_RFM['R'].map(str)) + np.array(df_RFM['F'].map(str))
df_RFM['Segment'] = df_RFM['Segment'].replace(segt_map, regex=True)
df_RFM.sample(10)

In [ ]:
df_RFM[df_RFM['Segment']=='нужно внимание'] #!!! Изменяйте параметр фильтра

In [ ]:
segments_counts = df_RFM['Segment'].value_counts().sort_values(ascending=True)

fig, ax = plt.subplots()

bars = ax.barh(range(len(segments_counts)),
              segments_counts,
              color='silver')
ax.set_frame_on(False)
ax.tick_params(left=False,
               bottom=False,
               labelbottom=False)
ax.set_yticks(range(len(segments_counts)))
ax.set_yticklabels(segments_counts.index)

for i, bar in enumerate(bars):
        value = bar.get_width()
        if segments_counts.index[i] in ['чемпионы', 'лояльные']:
            bar.set_color('firebrick')
        ax.text(value,
                bar.get_y() + bar.get_height()/2,
                '{:,} ({:}%)'.format(int(value),
                                   int(value*100/segments_counts.sum())),
                va='center',
                ha='left'
               )

plt.show()

# Кластеризация

Кластерный анализ — многомерная статистическая процедура, выполняющая сбор данных, содержащих информацию о выборке объектов, и затем упорядочивающая объекты в сравнительно однородные группы. Задача кластеризации относится к статистической обработке, а также к широкому классу задач обучения без учителя.

In [ ]:
df_clients['ProfitPct']=df_clients['Profit']/df_clients['Revenue']
df_clients.sample()

Ниже указываем на основе каких данных будет выполнять кластеризацию. ***Список столбцов можете изменять! Количество может быть произвольным***

In [ ]:
data_for_clust=df_clients[['Revenue', 'ProfitPct', 'Total_sale', 'Days_long']].copy()
data_for_clust.fillna(0, inplace=True)

In [ ]:
from sklearn import preprocessing
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import *
from matplotlib import rc
from sklearn.cluster import KMeans

dataNorm = preprocessing.normalize(data_for_clust) #нормализация данных

In [ ]:
# Метод локтя. Позволячет оценить оптимальное количество сегментов.
# Показывает сумму внутри групповых вариаций

data_dist = pdist(dataNorm, 'euclidean')
data_linkage = linkage(data_dist, method='average')

last = data_linkage[-10:, 2]
last_rev = last[::-1]
idxs = np.arange(1, len(last) + 1)
plt.plot(idxs, last_rev)

acceleration = np.diff(last, 2)  
acceleration_rev = acceleration[::-1]
plt.plot(idxs[:-2] + 1, acceleration_rev)
plt.show()
k = acceleration_rev.argmax() + 2 
print("рекомендовано кластеров:", k)

`n_clusters` - параметр, который указвает на какое количество кластеров делить выборку.

In [ ]:
km_m = KMeans(n_clusters=8, random_state=46).fit(dataNorm) ### !!! Изменяйте количество кластеров
data_for_clust['group_no']=km_m.labels_+1

In [ ]:
# основные статистики по нашим кластерам
df_clust=data_for_clust.groupby('group_no').median()
df_clust['Count']=data_for_clust.groupby('group_no')['group_no'].count()
df_clust

# Пожизненная ценность клиента

На основе анализа кластеров выполним оценку пожизненной ценности клиентов из каждого кластера.

Найдем коэффициенты удержания клиентов каждого кластера по годам.

In [ ]:
df_clients['group_no']=km_m.labels_+1

ya=df_clients['start_year'].unique()
ya.sort()
churn={}
for gr in df_clients['group_no'].unique():
    churn[gr]={}
    for y in range(1, len(ya)):
        try:
            churn[gr][ya[y]]=round(len(df_clients[(df_clients['last_year']>=ya[y]) 
                                            & (df_clients['group_no']==gr) &
                                                (df_clients['start_year']<ya[y])])/len(
                df_clients[(df_clients['start_year']<ya[y]) & ((df_clients['group_no']==gr))]),2)
        except:
            churn[gr][ya[y]]=0
            
df_churn=pd.DataFrame.from_dict(churn, orient='index')
df_churn

Чаще всего, коэффициенты удержания достаточно стабильны по сегментам. Если не происходит катострофы. И это один из важнейших показателей маркетинга/продаж на стагнирующем и тем более падающем рынке. К тому же, этот коэффициент позволяет оценить пожизненную ценность клиента для компании, те наиболее вероятный доход среднего клиента из сегмента. Рассчитывается по формуле:

**CLV=m*(r/(1+i-r))**

Где,

CLV (Customer Livetime Value) пожизнеснная ценность клиента
- m средняя маржинальная прибыль на клиента в год
- r коэффициент удержания для сегмента
- i ставка дисконтирования. Равна стоимости денег для компании плюс воспринимаемый риск. Учитывается так как считаем доход от клиента на горизонте более года. В нашем случае, пример равной 20%.

In [ ]:
discount=0.2 # !!! Можно изменять ставку дисконтирования
CLV={}
for gr in df_churn.index:
    CLV[gr]={}
    for y in df_churn.columns:
        CLV[gr][y]=round(df_clients[df_clients['group_no']==gr]['Profit'].median()*df_churn.loc[gr, y]/(1-discount+df_churn.loc[gr, y]),2)
df_CLV=pd.DataFrame.from_dict(CLV, orient='index')
df_CLV

In [ ]:
df_CLV.plot.bar(figsize=(14,4), title='CLV по годам в рамках каждого кластера');

# Предсказание оттока

Решим классическую задачу с попыткой предсказать вернется ли к нам клиент за следующей покупкой. Для решения задачи классификации используем алгоритм RandomForest.

Подготовим данные. Рассчитаем среднее количество дней между покупками клиентов (у кого было больше трех покупок).

In [ ]:
df_clients['DaysB']=df_clients[df_clients['Total_sale']>3]['Days_long']/df_clients[df_clients['Total_sale']>3]['Total_sale']
df_clients.describe()

In [ ]:
df_clients['DaysB'].hist(bins=100, figsize=(18,4));

In [ ]:
df_clients.loc[df_clients['Total_sale']<=3, 'DaysB']=25 #!!! Можно изменять! Тем у кого не было покупки устанавливаем интервал принудительно 

#Churn_day - дата, после которой считаем клиента потерянным
for i in df_clients.index:
    df_clients.loc[i, 'Churn_day']=df_clients.loc[i, 'last_date']+pd.Timedelta("P%sDT0H0M0.000000" % (round(df_clients.loc[i, 'DaysB']*2,0)))
try:
    df.drop(columns=['Churn_day', 'last_date'], inplace=True)
except:
    pass
    
df=df.merge(df_clients[['Churn_day','last_date', 'Name']], left_on='Name', right_on='Name')

df.sample(2)

Проставим признак `Next` - будет ли следующая покупка. Где 1- покупка будет.

In [ ]:
df['Next']=df.apply(lambda x: 1 if x['Date'] < x['last_date'] else 0, axis=1)
df['Next']=df.apply(lambda x: 0 if (x['Churn_day']<maxDate) and (x['Next']==0) else 1, axis=1)
df.describe()

***Продолжительная операция***

Создадим еще три параметра:
- SumCum накопленная сумма покупок
- CountCum количество предыдущих покупок
- FirstY год первой покупки

In [ ]:
from tqdm.notebook import tqdm

df.sort_values(by='Date', inplace=True)
df['SumCum']=0
df['CountCum']=0
df['FirstY']=0
for n in tqdm(df['Name'].unique()):
    for d in df[df['Name']==n].index:
        df.loc[d, 'SumCum']=df[(df['Name']==n) & (df['Date']<df.loc[d, 'Date'])]['Revenue'].sum()
        df.loc[d, 'CountCum']=df[(df['Name']==n) & (df['Date']<df.loc[d, 'Date'])]['Revenue'].count()
        df.loc[d, 'FirstY']=df[df['Name']==n]['Date'].min().year

In [ ]:
df.corr()

In [ ]:
corr = df.corr()
plt.figure(figsize=(14, 14))
sns.heatmap(corr[(corr >= 0.3) | (corr <= -0.3)],
            cmap="RdBu_r", vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
plt.figure(figsize=(6, 10))
sns.scatterplot(data=df, x='SumCum', y='CountCum', hue='Next');

In [ ]:
# продолжительная операция!!!
sns.pairplot(df,height=3, hue='Next', diag_kind='kde');

In [ ]:
df.columns

In [ ]:
col=['Revenue', 'SumCum', 'CountCum', 'FirstY'] #!!! можно менять названия столбцов для модели!

In [ ]:
# импортируем библиотеки
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score

#обучим модель
model = RandomForestClassifier(n_estimators=5)
model.fit(df[col], df.Next)

In [ ]:
# точность модели
r2_score(model.predict(df[col]), df.Next)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f

In [ ]:
#удобнее отобразить на столбчатой диаграмме
d_first = len(col)
plt.figure(figsize=(8, 8))
plt.title("Значимость признака")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(col)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);

Другие метрики точности.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
conf_mat = confusion_matrix(df.Next, model.predict(df[col]))
ax = sns.heatmap(conf_mat, annot=True, fmt='g');
ax.set_xlabel('Предсказанные значения')
ax.set_ylabel('Актуальные значения');

In [ ]:
print(classification_report(df.Next, model.predict(df[col])))